In [12]:
import pandas as pd
from collections import defaultdict
import codecs
import spacy
import nltk
from nltk import ngrams
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import string
import collections

In [2]:
def readXLS():
    #io = r'C:\\File\\CIS-668\\Project\\corpus-final09.xls' #Bohao local path
    io = r'corpus-final09.xls'
    df = pd.read_excel(io,sheet_name = 1)
    df = df.drop(['Group','Person','Native English','Knowledge','Difficulty'],axis=1)
    return df



def readFile(filename):
#     with codecs.open('C:\\File\\CIS-668\\Project\\corpu|s-final09\\'+filename,'r', encoding='utf-8', 
#                  errors='ignore') as f: # Bohao Path
    with codecs.open('C:\\Users\\Yogesh\\plagiarism-detection\\corpus\\'+filename,'r', encoding='utf-8', 
                 errors='ignore') as f:
         str = f.read() 
    return str
#readFile('orig_taska.txt')


def read_ori_files():
    files = defaultdict(str)   ##original files
    prefix = 'orig_task'
    for i in range(5):
        text = readFile(prefix+chr(97+i)+'.txt')
        #text = text.decode('utf8')
        if text:
            files[prefix+chr(97+i)+'.txt'] = text
    return files

def read_sus_files():
    df = readXLS()
    sus_dict = defaultdict(str)   # suspecious files
    for idx in range(len(df)):
        filename = df.iloc[idx][0]
        text = readFile(filename)
        sus_dict[filename] = text
    return sus_dict
        

In [3]:
# param type: string
# return type : list
def text_segmentation(text):
    sent_tokenized = sent_tokenize(text)
    return sent_tokenized

# param type: list
# reuturn type string
def text_tokenization(text):
    word_tokenized = []
    for sentence in text:
        punc_removal = remove_punctuation(word_tokenize(sentence))
        word_tokenized.append(punc_removal)
    return word_tokenized

# param type: string
# return type: string
def text_lowercase(text):
    return text.lower()

def remove_punctuation(sentence):
    result = []
    for token in sentence:
        if token not in string.punctuation:
            result.append(token)
    return result

def pre_processing(text):
    text = text_lowercase(text)
    #print(text)
    seg = text_segmentation(text)
    tokens = text_tokenization(seg)
    return tokens

In [27]:
def LCS_document(ori_list,sus_list):
    feature = {}
    length_s = len(sus_list)
    length_o = len(ori_list)
    sum_ =0
    lcs_len = []
    sus_max = []# to find the max for a given specious sentence
    sus_num = []
    for idx1 in range(length_s):
        sus_num = []
        for idx2 in range(length_o):
            nums = LCS_sentence(ori_list[idx2],sus_list[idx1])
            sus_num.append(nums)
            sum_ +=nums
            lcs_len.append(nums)
        if len(sus_num) > 1:
            temp = max(sus_num)
            sus_max.append(temp)
    if len(lcs_len) > 1:
        max_len = max(lcs_len)
    elif len(lcs_len) == 1:
        max_len = lcs_len[0]
    else:
        max_len = 0
    sus_sum_nol = 0
    if len(sus_max) > 1:
        sus_sum_nol = sum(sus_max) / length_s
    feature['longest matching sequence'] = max_len
    normalized_lcs = round(sus_sum_nol,4)
    average_matching_sequence =  round(sum_ / (length_s*length_o),4)
    feature['sum of longest matching sequence normalized by the number of suspecious sentence'] = normalized_lcs
    feature['average of matching sequence'] = average_matching_sequence
    
    if(similarity_score):
        return average_matching_sequence
    else:
        return feature

def LCS_sentence(original_tokens,suspecious_tokens):
    length_o = len(original_tokens)
    length_s = len(suspecious_tokens)
    LCS = [[0 for _ in range(length_s + 1)] for _ in range(length_o + 1)]
    res = []
    for i in range(1,length_o+1):
        for j in range(1,length_s+1):
            if original_tokens[i-1] == suspecious_tokens[j-1]:
                LCS[i][j] = LCS[i-1][j-1] + 1
            else:
                LCS[i][j] = max(LCS[i-1][j],LCS[i][j-1])
    return LCS[length_o][length_s]

In [4]:
def generate_ngrams(sentences,n):
    temp = serialize(sentences)
    #print(temp)
    trigrams =  nltk.ngrams(temp, n)
    trigram_arr = []
    for gram in trigrams:
        trigram_arr.append(gram)
    return trigram_arr

# functionality:flatten the list, converting it to one-dimension list
# param type:list[list]
# return type:list
def serialize(sentences):
    res = []
    if len(sentences) == 0:
        return []
    if len(sentences) == 1: # only one sentence
        return sentences
    for sent in sentences:
        for token in sent:
            res.append(token)
    return res

def jaccard_similarity(x,y):
    feature = {}
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = 1
    ferret = float(len(set.union(*[set(x), set(y)]))) #Ferret Comparison Technique(denominator is no of trigrams in two docs i.e. their union)
    containment =  float(len(set(y))) #Containment Measure technique(denominator is no of trigrams in suspicious docs)
    if(ferret == 0.0):
         feature['Ferret Trigram Similarity'] = 0
    else:
        feature['Ferret Trigram Similarity'] = intersection_cardinality / ferret
    if(containment == 0.0):
         #feature['Containment Trigram Similarity'] = 0
    else:
         #feature['Containment Trigram Similarity'] = intersection_cardinality / containment
   
    return feature

In [5]:

def dependancy_relations(original_text,suspicious_text):
    feature = {}
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(u"The Autonomous cars shift insurance liability toward manufacturers")
    original_doc = nlp(original_text)
    suspicious_doc = nlp(suspicious_text)
    original_doc_dependancies = []
    suspicious_doc_dependancies =[]
    for token in original_doc:
        original_doc_dependancies.append((token.text, token.head.text))
    for token in suspicious_doc:
        suspicious_doc_dependancies.append((token.text, token.head.text))
    
    #print(set(original_doc_dependancies))
    #print("--------------------------------\n\n")
    #print(set(suspicious_doc_dependancies))
    intersection_cardinality = len(set.intersection(*[set(original_doc_dependancies), set(suspicious_doc_dependancies)])) #common dependancies between original and suspicious text
    #print(intersection_cardinality)
    feature['dependancy_relations'] = intersection_cardinality / len(set(suspicious_doc_dependancies))
    return feature


In [14]:
def bigramlanguagemodel(inputtext):
    input = inputtext.split(' ')
    lis = list(input)
    count = collections.Counter()
    output = {}

    for i in range(len(input)-2+1):
        g = ' '.join(input[i:i+2])
        output.setdefault(g, 0)
        output[g] += 1

        
    bigramprob = {}
    for key, value in output.items():
        listicle = list(key.split())
        divisor = lis.count(listicle[0])
        if(divisor == 0):
            biprob = 0
        else:
            biprob = float(value)/lis.count(listicle[0])
        bigramprob.setdefault(key,0)
        bigramprob[key] = biprob
    return bigramprob

def bigram_scorer(otext,stext):
    feature = {}
    ogtext = set(otext)
    sptext = set(stext)
    score = 1.
    for name in ogtext.intersection(sptext):
        score *= stext[name]
    print(score)
    feature['dependancy_relations'] = score
    return feature
#scorer(bigramlanguagemodel(otext),bigramlanguagemodel(stext))  

In [56]:
def generate_featureset(techniques,similarity_score):
    ori_dict = read_ori_files()
    sus_dict = read_sus_files()
    sus_df = readXLS()
    feature_set = []
    for idx1 in range(5):
        token_ori = pre_processing(ori_dict['orig_task'+chr(97+idx1)+'.txt'])
        #print('orig_task'+chr(97+idx1)+'.txt')
        temp_df = sus_df[sus_df['Task'] == chr(97+idx1)]
        for index, row in temp_df.iterrows():
            text_name = row['File']
            cat = row['Category']
            token_sus = pre_processing(sus_dict[text_name])
            if("LCS" in techniques):   
                feature = LCS_document(token_ori,token_sus) # get the featrue sets for one file
                feature_set.append((feature,cat))
            if("Trigram Similarity" in techniques):
                feature = jaccard_similarity( generate_ngrams(token_ori,3),generate_ngrams(token_sus,3))
                feature_set.append((feature,cat))
            if("Dependancy Relations" in techniques):
                feature = dependancy_relations(ori_dict['orig_task'+chr(97+idx1)+'.txt'],sus_dict[text_name])
                feature_set.append((feature,cat))
            if("Language Model" in techniques):
                feature = bigram_scorer(bigramlanguagemodel(ori_dict['orig_task'+chr(97+idx1)+'.txt']),bigramlanguagemodel(sus_dict[text_name]))
                feature_set.append((feature,cat))
    return feature_set
    

In [57]:
# get trainset and test set
feature_set = generate_featureset(["LCS"],False)
trainset = feature_set[:80]
testset = feature_set[80:]
# machine learning
classifier = nltk.NaiveBayesClassifier.train(trainset)
print(nltk.classify.accuracy(classifier,testset))


TypeError: LCS_document() missing 1 required positional argument: 'similarity_score'

In [42]:
feature_set = generate_featureset(["Dependancy Relations"],False)


In [54]:
def similiarity_scores_by_category(feature_set, technique):
    non_score = []
    heavy_score = []
    light_score = []
    cut_score = []
    scores_by_category = {}
    for obs in feature_set:
        cat = obs[1]
        if(cat == "non"):
            non_score.append(obs[0][technique])
        if(cat == "light"):
                light_score.append(obs[0][technique])
        if(cat == "heavy"):
                heavy_score.append(obs[0][technique])
        if(cat == "cut"):
                cut_score.append(obs[0][technique])
    scores_by_category['non'] = list_avg(non_score)
    scores_by_category['light'] = list_avg(light_score)
    scores_by_category['heavy'] = list_avg(heavy_score)
    scores_by_category['cut'] = list_avg(cut_score)
    return scores_by_category
        
def list_avg(lst):
    return sum(lst)/len(lst)

In [55]:

scores = similiarity_scores_by_category(feature_set,"Ferret Trigram Similarity")
scores

{'non': 0.0070389247790770566,
 'light': 0.24885620650861529,
 'heavy': 0.1413698352275447,
 'cut': 0.3505865841003097}